In [1]:
import sys
from pathlib import Path

# Ubicación del notebook
NOTEBOOK_DIR = Path.cwd()

# Raíz del proyecto = subir un nivel desde Notebooks/
PROJECT_ROOT = NOTEBOOK_DIR.parent

# Añadir raíz del proyecto al sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Proyecto raíz detectado:", PROJECT_ROOT)

Proyecto raíz detectado: c:\Users\cathe\Didier Jesus\EduFinance\EduFinance_Simulator


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

from utils.paths import DATA_DIR, FIG_DIR
from utils.loader import load_csv

# Directorios
MODELS_DIR = PROJECT_ROOT / "models_results"
GARCH_RESULTS_DIR = MODELS_DIR  / "Garch_results"
GARCH_FIG_DIR = FIG_DIR / "Garch_fig_results"

GARCH_RESULTS_DIR.mkdir(parents=True, exist_ok=True)
GARCH_FIG_DIR.mkdir(parents=True, exist_ok=True)


In [3]:
log_diff = load_csv(DATA_DIR / "time_series" / "log_diff.csv")
log_diff.index = pd.to_datetime(log_diff.index)

tickers = [t for t in log_diff.columns if t.lower() != "date"]
len(log_diff), tickers


(1930, ['BTC-USD', 'EUNL.DE', 'QQQ', 'TSLA', 'V', 'VOO', 'XAR', 'XRP-USD'])

In [4]:
def evaluate_garch_model(series, p, q, train_size=30):

    try:
        train = series[:-train_size]
        valid = series[-train_size:]

        model = arch_model(
            train,
            mean="Constant",
            vol="GARCH",
            p=p,
            q=q,
            dist="normal"
        )
        res = model.fit(disp="off")

        # Forecast
        forecasts = res.forecast(horizon=train_size)
        pred_mean = forecasts.mean.iloc[-1].values
        pred_vol  = forecasts.variance.iloc[-1].values ** 0.5

        # Métricas
        rmse_ret = np.sqrt(mean_squared_error(valid, pred_mean))
        mae_ret = mean_absolute_error(valid, pred_mean)
        mape_ret = np.mean(np.abs((valid - pred_mean) / valid)) * 100

        rmse_vol = np.sqrt(mean_squared_error((valid**2)**0.5, pred_vol))

        return {
            "order": (p, q),
            "aic": res.aic,
            "bic": res.bic,
            "persistence": res.params["alpha[1]"] + res.params["beta[1]"],
            "rmse_ret": rmse_ret,
            "mae_ret": mae_ret,
            "mape_ret": mape_ret,
            "rmse_vol": rmse_vol,
            "valid": valid,
            "pred_mean": pred_mean,
            "pred_vol": pred_vol,
            "model": res
        }
    except:
        return None


In [7]:
def evaluate_safe_garch(series, train_size=30):

    try:
        train = series[:-train_size]
        valid = series[-train_size:]

        model = arch_model(
            train,
            mean="Constant",
            vol="GARCH",
            p=1,
            q=1,
            dist="normal"
        )
        res = model.fit(disp="off")

        fc = res.forecast(horizon=train_size)
        pred_mean = fc.mean.iloc[-1].values
        pred_vol  = np.sqrt(fc.variance.iloc[-1].values)

        rmse_ret = np.sqrt(mean_squared_error(valid, pred_mean))
        mae_ret = mean_absolute_error(valid, pred_mean)
        mape_ret = np.mean(np.abs((valid - pred_mean) / valid)) * 100
        rmse_vol = np.sqrt(mean_squared_error(np.abs(valid), pred_vol))

        return {
            "order": (1, 1),
            "aic": res.aic,
            "bic": res.bic,
            "persistence": res.params["alpha[1]"] + res.params["beta[1]"],
            "rmse_ret": rmse_ret,
            "mae_ret": mae_ret,
            "mape_ret": mape_ret,
            "rmse_vol": rmse_vol,
            "valid": valid,
            "pred_mean": pred_mean,
            "pred_vol": pred_vol,
            "model": res
        }

    except:
        return None


In [8]:
p_values = [1, 2]
q_values = [1, 2]

garch_best_models = []
best_models = {}


In [9]:
for tk in tickers:
    print(f"\n=== Procesando {tk} ===")
    series = log_diff[tk].dropna()

    best_score = np.inf
    best_model_data = None

    # GRID SEARCH
    for p in p_values:
        for q in q_values:

            result = evaluate_garch_model(series, p, q)

            if result is None:
                continue

            # criterio de selección: MAPE del retorno
            if result["mape_ret"] < best_score:
                best_score = result["mape_ret"]
                best_model_data = result

    # FALLBACK: ningún modelo funcionó
    if best_model_data is None:
        print(f"Ningún modelo GARCH válido encontrado para {tk}. Intentando GARCH(1,1)...")

        fallback = evaluate_safe_garch(series)

        if fallback is None:
            print(f"{tk} no pudo ser modelado ni siquiera con GARCH(1,1). Lo omitimos.")
            continue

        print(f"✔ Se usará el modelo alternativo GARCH(1,1) para {tk}.")
        best_model_data = fallback

    # Guardar mejor modelo del ticker
    best_models[tk] = best_model_data
    garch_best_models.append({
        "ticker": tk,
        "order": best_model_data["order"],
        "aic": best_model_data["aic"],
        "bic": best_model_data["bic"],
        "persistence": best_model_data["persistence"],
        "rmse_vol": best_model_data["rmse_vol"],
        "rmse_ret": best_model_data["rmse_ret"],
        "mape_ret": best_model_data["mape_ret"],
        "train_size": len(series)-30,
        "valid_size": 30
    })

    print(f" Mejor modelo: GARCH{best_model_data['order']}  | MAPE_ret={best_model_data['mape_ret']:.3f}%")




=== Procesando BTC-USD ===
 Mejor modelo: GARCH(2, 2)  | MAPE_ret=365.391%

=== Procesando EUNL.DE ===
Ningún modelo GARCH válido encontrado para EUNL.DE. Intentando GARCH(1,1)...
✔ Se usará el modelo alternativo GARCH(1,1) para EUNL.DE.
 Mejor modelo: GARCH(1, 1)  | MAPE_ret=inf%

=== Procesando QQQ ===
 Mejor modelo: GARCH(2, 1)  | MAPE_ret=110.428%

=== Procesando TSLA ===
 Mejor modelo: GARCH(1, 1)  | MAPE_ret=104.323%

=== Procesando V ===
 Mejor modelo: GARCH(1, 1)  | MAPE_ret=114.312%

=== Procesando VOO ===
 Mejor modelo: GARCH(2, 2)  | MAPE_ret=323.465%

=== Procesando XAR ===
 Mejor modelo: GARCH(2, 2)  | MAPE_ret=97.087%

=== Procesando XRP-USD ===
 Mejor modelo: GARCH(1, 2)  | MAPE_ret=124.005%


In [10]:
garch_metrics_df = pd.DataFrame(garch_best_models).sort_values("mape_ret")
garch_metrics_df.to_csv(GARCH_RESULTS_DIR / "GARCH_all_metrics.csv", index=False)
garch_metrics_df


,ticker,order,aic,bic,persistence,rmse_vol,rmse_ret,mape_ret,train_size,valid_size
6,XAR,"(2, 2)",-10849.335744,-10816.038089,0.490000,0.009556,0.009238,97.086849,1900,30
3,TSLA,"(1, 1)",-6949.520374,-6927.321937,0.980000,0.023172,0.026929,104.323208,1900,30
2,QQQ,"(2, 1)",-11072.041294,-11044.293248,0.930000,0.006302,0.008474,110.428255,1900,30
4,V,"(1, 1)",-10742.451821,-10720.253384,0.980000,0.008455,0.009617,114.312258,1900,30
7,XRP-USD,"(1, 2)",-5214.311617,-5186.563572,0.265940,0.056792,0.039417,124.005097,1900,30
5,VOO,"(2, 2)",-12194.293018,-12160.995363,0.490000,0.004886,0.006613,323.465339,1900,30
0,BTC-USD,"(2, 2)",-6770.501964,-6737.204309,0.837367,0.025859,0.021504,365.390824,1900,30
1,EUNL.DE,"(1, 1)",-12552.033140,-12529.834704,0.900000,0.007375,0.007536,inf,1900,30


In [14]:
for tk, model in best_models.items():

    valid_idx = model["valid"].index    # fechas reales
    pred_mean = pd.Series(model["pred_mean"], index=valid_idx)
    pred_vol  = pd.Series(model["pred_vol"], index=valid_idx)

    df_comp = pd.DataFrame({
        "actual_return": model["valid"],
        "predicted_return": pred_mean,
        "predicted_volatility": pred_vol,
    })

    df_comp["abs_error_ret"] = np.abs(df_comp["actual_return"] - df_comp["predicted_return"])
    df_comp["mape_ret"] = (df_comp["abs_error_ret"] / df_comp["actual_return"]) * 100

    df_comp.to_csv(GARCH_RESULTS_DIR / f"{tk}_garch_comparison.csv")

In [13]:
for tk, model in best_models.items():

    series = log_diff[tk].dropna()

    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(series, label="Retornos", alpha=0.55)
    ax.plot(model["model"].conditional_volatility, label="Volatilidad condicional", color="tomato")
    ax.set_title(f"{tk} — Mejor GARCH {model['order']}")
    ax.legend()
    ax.grid(True)

    fig.savefig(GARCH_FIG_DIR / f"{tk}_volatility.png", dpi=300)
    plt.close()
